1. 损失出现在最后，后面的层训练较快
2. 数据在最底层
    底部的层训练较慢
    底部层一变化，所有都得跟着变
    最后的那些层需要重新学习很多次
    导致收敛变慢

批量归一化固定小批量中的均值和方差，然后学习出适合的偏移和缩放
可以加速收敛速度，但一般不改变模型精度

In [1]:
import torch
from torch import nn
from d2l import torch as d2l

In [2]:
# moving_mean, moving_var是全局的均值和方差
def batch_norm(X, gamma, beta, moving_mean, moving_var, eps, momentum):
    # 通过is_grad_enabled方法来判断当前模式是训练模式还是预测模式
    if not torch.is_grad_enabled():
        # 在inference
        X_hat = (X - moving_mean) / torch.sqrt(moving_var + eps)
    else:
        # 这里是间接实现，2是全连接层， 4是2D卷积层
        assert len(X.shape) in (2, 4)
        if len(X.shape) == 2:
            # 按照行求
            mean = X.mean(dim=0)
            var = ((X - mean) ** 2).mean(dim=0)
        else:
            # 1*n*1*1
            mean = X.mean(dim=(0, 2, 3), keepdim=True)
            var = ((X - mean) ** 2).mean(dim=(0, 2, 3), keepdim=True)
        # 训练模式下，用当前的均值和方差做标准化
        X_hat = (X - mean) / torch.sqrt(var + eps)
        # 更新移动平均的均值和方差
        moving_mean = momentum * moving_mean + (1.0 - momentum) * mean
        moving_var = momentum * moving_var + (1.0 - momentum) * var
    # 缩放和位移
    Y = gamma * X_hat + beta
    return Y, moving_mean.data, moving_var.data